# Data Creation
In this first notebook we are going to:<br>
1. Extract subgraphs from the 6 Neo4j databases
2. Transform the subgraphs in text form
3. Generate the texts based on the triples
4. Format triples and texts in order to make them usable for finetuning

## Neo4j Extractor
This is the first notebook, used for extracting triples from the 6 neo4j databases. The triples are going to be extracted by selecting a rendom node, and then identifying a subgraph with random maxLevel and limit starting from that node.<br>
Since the database has to be stopped and restarted manually, this notebook was run for each of the 6 databases by changing the db_name and the number of samples to extract from each of them.

In [ ]:
# Import the libraries
from neo4j import GraphDatabase
import random
import re
from tqdm import tqdm

## Define the name of the database and the number of subgraph to extract from it
db_name = "graph-data-science"
num_samples = 200

# Neo4j connection details
uri = "bolt://localhost:7687"  # Update with your Neo4j URI
user = "neo4j"  # Update with your Neo4j username
password = "*****"  # Update with your Neo4j password

# Connect to Neo4j
driver = GraphDatabase.driver(uri, auth=(user, password))

# Function to extract all node ids inside the database
def get_all_node_ids(tx):
    result = tx.run("MATCH (n) RETURN ID(n) AS id")
    return [record["id"] for record in result]

# Step 1: Retrieve all node IDs
with driver.session() as session:
    all_node_ids = session.execute_read(get_all_node_ids)

# Step 2: Randomly select node IDs
sampled_node_ids = random.sample(all_node_ids, num_samples)

In [ ]:
# The coalesceDiz is used for specifying the properties of the nodes that have to be extracted from the database. For each node, the first property of the list that is not null in the node is going to be extracted.
coalesceDiz = {
    "recommendations": ["name", "title"],
    "graph-data-science": ["name"],
    "legis-graph": ["wikipediaID", "name", "title", "code", "billID", "type"],
    "wwc2019": ["name"],
    "twitch": ["name"],
    "recipes": ["name"],
    "listings": ["name"]
}

In [ ]:
# Define the query used to extract the subgraph from the database
# The query has 3 parameters:
# 1. The node id that we extracted before from which to start
# 2. The maxLevel, i.e. the depth of the subgraph
# 3. Limit, i.e. the maximum number of nodes
query = """MATCH (n)
WHERE ID(n) = $node_id
CALL apoc.path.subgraphAll(n, {
    maxLevel: $maxLevel, limit: $limit, bfs: False
}) YIELD nodes, relationships
UNWIND relationships AS r
WITH r, startNode(r) AS startNode, endNode(r) AS endNode
RETURN
    coalesce("""

# Insert the corresponding coalesce information in the query from the coalesceDiz
for entity in coalesceDiz[db_name]:
    query += "startNode." + entity + ","
query += """ 'unknown') AS startNodeLabel,
    type(r) AS relationshipType,
    coalesce("""
for entity in coalesceDiz[db_name]:
    query += "endNode." + entity + ","
query += " 'unknown') AS endNodeLabel"

# Final query
print(query)

MATCH (n)
WHERE ID(n) = $node_id
CALL apoc.path.subgraphAll(n, {
    maxLevel: $maxLevel, limit: $limit, bfs: False
}) YIELD nodes, relationships
UNWIND relationships AS r
WITH r, startNode(r) AS startNode, endNode(r) AS endNode
RETURN 
    coalesce(startNode.name, 'unknown') AS startNodeLabel, 
    type(r) AS relationshipType,
    coalesce(endNode.name, 'unknown') AS endNodeLabel


In [ ]:
# For some movies in the recommendations dataset I noticed that the article was moved to the end (ex: "Gentlmen The")
# Since this could be a problem when generating a text that contains this strange titles, we are going to solve this with regex
def move_article_to_beginning(title: str) -> str:
    # Use regex to match the pattern and rearrange the string
    result = re.sub(r'^(.*), The$', r'The \1', title)
    return result

# Function to execute the query and extract the subgraph
def get_subgraph(node_id, maxLevel, limit):
    driver = GraphDatabase.driver(uri, auth=(user, password))
    try:
        with driver.session() as session:
            result = session.run(query, node_id=node_id, maxLevel=maxLevel, limit=limit)
            return result.values()
    finally:
        driver.close()

# Transform the extracted subgraph in text form
def build_prompt(subgraph):
    prompt = ""
    for triple in subgraph:
        triple[0] = move_article_to_beginning(triple[0]).strip()
        triple[2] = move_article_to_beginning(triple[2]).strip()
        prompt += "(" + re.sub("[(].*[)]","",triple[0]) + ") - [" + triple[1] + "] -> (" + re.sub("[(].*[)]","",triple[2]) + ")\n"
    return prompt.strip()

# Run the pipeline
for id in tqdm(sampled_node_ids):
    # Choose a random maxLevel and limit
    maxLevel = random.randrange(2,6)
    limit = random.randrange(6,13)
    # Extract subgraph by running the query
    subG = get_subgraph(id, maxLevel, limit)
    # Transform the extracted subgraph in text form
    prompt = build_prompt(subG)
    # Write to file
    f = open("C:/Users/david/Desktop/NLP/data/triples/" + db_name + "/" + str(id) + ".txt", "a")
    f.write(prompt)
    f.close()

100%|██████████| 200/200 [06:51<00:00,  2.06s/it]


## Text Generation
Now that the triples are extract we generate, starting from those triples, the texts.<br>
To do so, we are going to use the gemini-pro API.

In [ ]:
import os
from tqdm import tqdm
import random
import re
import google.generativeai as genai

c:\Users\david\anaconda3\envs\NLPenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
genai.configure(api_key="***")


In [ ]:
model = genai.GenerativeModel('gemini-pro')

In [ ]:
input_path = "C:/Users/david/Desktop/NLP/data/triples/"
output_path = "C:/Users/david/Desktop/NLP/data/texts/"

In [ ]:
# This dictionary contains, for each database, the explanation of all of its relationships.
# The explanations of the relationships present between the triples are going to be inserted inside the prompt in order to supply to the LLM a better understanding of them and hopefully generating better text.
triplets_instructions = {
    "recommendations": {
        "IN_GENRE": "(Movie)-[IN_GENRE]->(Genre)",
        "ACTED_IN": "(Person/Director/Actor)-[ACTED_IN]->(Movie)",
        "DIRECTED": "(Person/Director/Actor)-[DIRECTED]->(Movie)",
        "RATED": "(User)-[RATED]->(Movie)"
    },
    "graph-data-science": {
        "ATTACKER": "(House)-[ATTACKER, DEFENDER]->(Battle)",
        "DEFENDER": "(House)-[ATTACKER, DEFENDER]->(Battle)",
        "IS_IN": "(Battle, Location)-[IS_IN]->(Location, Region)",
        "DEFENDER_COMMANDER": "(Knight, King)-[DEFENDER_COMMANDER,ATTACKER_COMMANDER]->(Battle)",
        "ATTACKER_COMMANDER": "(Knight, King)-[DEFENDER_COMMANDER,ATTACKER_COMMANDER]->(Battle)",
        "DEFENDER_KING": "(King)-[DEFENDER_KING, ATTACKER_KING]->(Battle)",
        "ATTACKER_KING": "(King)-[DEFENDER_KING, ATTACKER_KING]->(Battle)",
        "BELONGS_TO": "(Person,Knight, King)-[BELONGS_TO]->(House)",
        "HAS_STATUS": "(Person,Knight, King)-[HAS_STATUS]->(Status)",
        "APPEARED_IN": "(Person,Knight,King)-[APPEARED_IN, DIED_IN]->(Book)",
        "DIED_IN": "(Person,Knight,King)-[APPEARED_IN, DIED_IN]->(Book)",
        "MEMBER_OF_CULTURE": "(Person, King, Knight)-[MEMBER_OF_CULTURE]->(Culture)",
        "INTERACTS": "(Person, Knight, King)-[INTERACTS]->(Person, Knight, King)",
        "RELATED_TO": "(Person, Knight, King)<-[RELATED_TO]->(Person, Knight, King)"
    },
    "legis-graph": {
        "REPRESENTS": "(Legislator)-[REPRESENTS]->(StateCode)",
        "IS_MEMBER_OF": "(Legislator)-[IS_MEMBER_OF]->(Party)",
        "ELECTED_TO": "(Legislator)-[ELECTED_TO]->(Body)",
        "SPONSORED_BY": "(BillId)-[SPONSORED_BY]->(Legislator)",
        "VOTED_ON": "(Legislator)-[VOTED_ON]->(BillId)",
        "REFERRED_TO": "(BillId)-[REFERRED_TO]->(Committee)",
        "SERVES_ON": "(Legislator)-[SERVES_ON]->(Committee)",
        "DEALS_WITH": "(BillId)-[DEALS_WITH]->(Subject)"
    },
    "wwc2019": {
        "PARTICIPATED_IN": "(Team)-[PARTICIPATED_IN]->(Tournament)",
        "PLAYED_IN": "(Person)-[PLAYED_IN]->(Tournament)",
        "REPRESENTS": "(Person)-[REPRESENTS]->(Team)"
    },
    "recipes": {
        "WROTE": "(Author)-[WROTE]->(Recipe)",
        "CONTAINS_INGREDIENT": "(Recipe)-[CONTAINS_INGREDIENT]->(Ingredient)",
        "DIET_TYPE": "(Recipe)-[DIET_TYPE]->(DietType)",
        "COLLECTION": "(Recipe)-[COLLECTION]->(Collection)"
    },
    "listings": {
        "IN_NEIGHBORHOOD": "(ListingTitle)-[IN_NEIGHBORHOOD]->(Neighborhood)",
        "HAS": "(ListingTitle)-[HAS]->(Amenity)",
        "HOSTS": "(Host)-[HOSTS]->(ListingTitle)",
        "REVIEWED": "(User)-[REVIEWED]->(ListingTitle)"
    }
}

# This is the base system prompt
base_system_prompt = """
Imagine being a text generator from Knowledge Graphs.
Based on the triples provided in the context, generate a short text containing all the information contained in the triples.
Make sure not to add any information of the entities mentioned in the triples that is not coming from the knowledge graph.
Even though the usage of pronouns is allowed, make sure not to modify the names of the entities.
The text you generate should not be a simple mention of all the facts stored in the triples, but you should write them in an original way.
The text should resemble a """

# The style of the prompt is going to be randomly picked from this list
styles = ["blog article.", "wikipedia article.", "newspaper article.", "reddit post.", "YouTube script.", "podcast transcript."]

# These are specific instructions for some dbs
db_specific_instructions = {
    "recipes": "Insert the recipes' titles inside quotes.",
    "listings": "Insert the listings' titles inside quotes."
}

In [ ]:
# Function to generate the prompt
def generateSystemPrompt(rel_types):
    # Pick a random style from the list
    style_num = random.randrange(0,6)
    # Attach the random style to the base system prompt
    system_prompt = base_system_prompt + styles[style_num]
    # Add db specific instruction if needed
    if db_name in db_specific_instructions.keys():
        system_prompt += "\n" + db_specific_instructions[db_name]
    # Add db structure
    system_prompt += "\n\nThis is the KB structure:"
    triplets_lst = []
    for el in rel_types:
        triplets_lst.append(triplets_instructions[db_name][el])
    triplets_lst = list(set(triplets_lst))
    for el in triplets_lst:
        system_prompt += "\n" + el
    system_prompt += "\n\n Context:"
    return system_prompt

# Run the code for each of the db
# (This function was later run also for the other 2)
for db_name in ["recommendations", "legis-graph", "recipes", "listings"]:
    file_lst = os.listdir(input_path + db_name)
    file_lst = list(set(file_lst).difference(os.listdir(output_path + db_name)))
    # Loop throught the triples files
    for file_name in tqdm(file_lst):
        # Read file
        file_path = db_name + "/" + file_name
        file = open(input_path + file_path, "r")
        triples = file.read()
        # Extract the relationships types in the file
        rel_types = list(set(re.findall(r"\[(.*)\]", triples)))
        # Generate prompt
        system_prompt = generateSystemPrompt(rel_types)
        # Concatenate context triples
        final_prompt = system_prompt + '\n\n' + triples
        # Inference with Gemini Pro
        try:
            response = model.generate_content(final_prompt).text
        except:
            continue
        # Write the file to the output_path
        f = open(output_path + file_path, "a")
        f.write(response)
        f.close()

100%|██████████| 450/450 [25:52<00:00,  3.45s/it]


## Formatter
Now that we have both the triples and the text, we are going to format it in order to make it usable for finetuning the LLM.

In [ ]:
import os
import re
from tqdm import tqdm
import random
import pandas as pd

In [ ]:
# The schema dictionary is used to insert inside the prompt the schema of the corresponding database, i.e. the schema that the FineTuned LLM will have to follow in order to extract the triples form the text.
schema = {
    "recommendations": """(Movie)-[IN_GENRE]->(Genre)
(Person/Director/Actor)-[ACTED_IN]->(Movie)
(Person/Director/Actor)-[DIRECTED]->(Movie)
(User)-[RATED]->(Movie)""",
    "graph-data-science": """(House)-[ATTACKER, DEFENDER]->(Battle)
(House)-[ATTACKER, DEFENDER]->(Battle)
(Battle, Location)-[IS_IN]->(Location, Region)
(Knight, King)-[DEFENDER_COMMANDER,ATTACKER_COMMANDER]->(Battle)
(Knight, King)-[DEFENDER_COMMANDER,ATTACKER_COMMANDER]->(Battle)
(King)-[DEFENDER_KING, ATTACKER_KING]->(Battle)
(King)-[DEFENDER_KING, ATTACKER_KING]->(Battle)
(Person,Knight, King)-[BELONGS_TO]->(House)
(Person,Knight, King)-[HAS_STATUS]->(Status)
(Person,Knight,King)-[APPEARED_IN, DIED_IN]->(Book)
(Person,Knight,King)-[APPEARED_IN, DIED_IN]->(Book)
(Person, King, Knight)-[MEMBER_OF_CULTURE]->(Culture)
(Person, Knight, King)-[INTERACTS]->(Person, Knight, King)
(Person, Knight, King)<-[RELATED_TO]->(Person, Knight, King)""",
    "legis-graph": """(Legislator)-[REPRESENTS]->(StateCode)
(Legislator)-[IS_MEMBER_OF]->(Party)
(Legislator)-[ELECTED_TO]->(Body)
(BillId)-[SPONSORED_BY]->(Legislator)
(Legislator)-[VOTED_ON]->(BillId)
(BillId)-[REFERRED_TO]->(Committee)
(Legislator)-[SERVES_ON]->(Committee)
(BillId)-[DEALS_WITH]->(Subject)""",
    "wwc2019": """(Team)-[PARTICIPATED_IN]->(Tournament)
(Person)-[PLAYED_IN]->(Tournament)
(Person)-[REPRESENTS]->(Team)""",
    "recipes": """(Author)-[WROTE]->(Recipe)
(Recipe)-[CONTAINS_INGREDIENT]->(Ingredient)
(Recipe)-[DIET_TYPE]->(DietType)
(Recipe)-[COLLECTION]->(Collection)""",
    "listings": """(ListingTitle)-[IN_NEIGHBORHOOD]->(Neighborhood)
(ListingTitle)-[HAS]->(Amenity)
(Host)-[HOSTS]->(ListingTitle)
(User)-[REVIEWED]->(ListingTitle)"""
}

base_prompt = """<s>[INST]Imagine being a Knowledge Graph constructor from unstructured text.
Following the schema provided, extract all the triples you can find in the text.

Schema:
"""

In [ ]:
triples_path = "C:/Users/david/Desktop/NLP/data/triples/"
text_path = "C:/Users/david/Desktop/NLP/data/texts/"
output_path = "C:/Users/david/Desktop/NLP/data/train4/"

In [ ]:
# For each of the four databases used for training
for db_name in os.listdir(output_path):
    # Concatenate the base prompt with the schema of the relevant db
    db_prompt = base_prompt + schema[db_name] + '\n\nContext:\n"'
    # For each of the triple/text file:
    for file_name in tqdm(os.listdir(text_path + db_name)):
        # Read text file
        text = open(text_path + db_name + '/' + file_name, "r").read()
        text = "\n".join(text.split("\n\n"))
        # Append text file to prompt
        final_prompt = db_prompt + text + '"[/INST]\n\nExtracted Triples:\n'
        # Read triple file
        try:
            triples = open(triples_path + db_name + '/' + file_name, "r").read()
        except:
            print("Could not find triples of " + db_name + '/' + file_name)
            continue
        # Append triples to prompt
        final_prompt += triples
        final_prompt += "</s>"
        # Write the prompt into the output path
        f = open(output_path + db_name + '/' + file_name, "w")
        f.write(final_prompt)
        f.close()

100%|██████████| 986/986 [00:09<00:00, 102.86it/s]


Now that everything is saved, we open the files and put them into our datasets.

In [ ]:
# Insert everything into a list
lst = []
for db_name in os.listdir(output_path):
    if db_name == 'datasets':
        continue
    listdir = os.listdir(output_path + db_name)
    for file in listdir:
        text = open(output_path + db_name + '/' + file, "r").read()
        lst.append(text)

In [ ]:
# Shuffle the list
random.shuffle(lst)

In [ ]:
# Divide in train, validation, and test (2400, 400, 400)
# Note that we can slice the list in this way, and not randomly, because we shuffled it previously
df_train = pd.DataFrame(lst[:2400])
df_val = pd.DataFrame(lst[2400:2800])
df_test = pd.DataFrame(lst[-400:])

In [ ]:
# Save train and validation into parquet format (and manually upload to huggingFace: Giardooo/KG_constructor)
table = pa.Table.from_pandas(df_train)
pq.write_table(table, 'train-00000-of-00001.parquet')
table = pa.Table.from_pandas(df_val)
pq.write_table(table, 'test-00000-of-00001.parquet')

In [ ]:
# Save test file to csv
df_test.to_csv("test.csv")

The same pipeline was applied for the two external databases, resulting in a 210 observation datasets saved as "test_external_dbs.csv".<br>
The observations are 210 because 10 of them (5 for each database) will be used as examples in the few shot prompting technique, while the other 200 (100 per database) for testing.

# Data Preparation for Few Shot Finetuning
In this section we are going to create the dataset that is going to be used to finetune a model over prompts that contain examples.<br>
We are going to do this by importing the previous dataset, and inserting in each of the observation three other random observations coming from the same dataset and related to the same KG structure as the text from which the LLM will have to extract the triples.<br>
We used three examples rather than five to keep the observations shorter. In fact, when trying to fine-tune the LLM with prompts containing 5 examples, we stumbled upon OOM errors.

## Import libraries and data

In [ ]:
import random as rd

In [ ]:
from datasets import load_dataset
# Load Dataset
data = load_dataset("Giardooo/KG_constructor")
print(data)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/2400 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['0'],
        num_rows: 2400
    })
    test: Dataset({
        features: ['0'],
        num_rows: 400
    })
})


## Classify Data
We build a classifier function that helps us classify the observations back into the datasets they were extracted from.<br>

In [ ]:
def classifier(text):
  # The element are classified based on their univoque relatinoship types
  if "DEALS_WITH" in text:
    return "legis_graph"
  elif "ACTED_IN" in text:
    return "recommendations"
  elif "COLLECTION" in text:
    return "recipes"
  elif "IN_NEIGHBORHOOD" in text:
    return "listings"
  else:
    print(text)

In [ ]:
# Initialize class_diz
# In this dictionary we are going to store, both for train and validation, the index of the observations coming from each of the original datasets.
class_diz = {"train": {"legis_graph": [], "recommendations": [], "recipes": [], "listings": []}, "val": {"legis_graph": [], "recommendations": [], "recipes": [], "listings": []}}

for i in range(len(data["train"]["0"])):
  el = data["train"]["0"][i]
  # Classify element
  classification =  classifier(el)
  class_diz["train"][classification].append(i)

for i in range(len(data["test"]["0"])):
  el = data["test"]["0"][i]
  # Classify element
  classification =  classifier(el)
  class_diz["val"][classification].append(i)

## Build Examples

In [ ]:
# The function format_examples takes in input a list of 3 indexes and the data from which to extract the observations
# It then formats the three observations coming from this 3 indexes in a way that is suitable for the prompt
def format_examples(extract, data):
  example_txt = ""
  for el in extract:
    example_txt += "\nContext:\n"
    example_txt += data[el].split("Context:\n")[-1]
    example_txt += "\n-------------"
  example_txt = example_txt.replace("[/INST]", "").replace("</s>", "")
  return example_txt

# The function select_examples takes in input the position of the observation that we want to augment with examples and the data from which to extract the observations
# It then uses the class_diz to extract three examples coming from the same original dataset of the observation
def select_examples(position, data, train_val):
  # Classify current element
  classification =  classifier(data[position])
  # Get list of text of the same class
  lst = class_diz[train_val][classification].copy()
  # Remove current element
  lst.remove(position)
  extract = rd.sample(lst, 3)
  # Now that we have our list, format the 3 examples
  return format_examples(extract, data)

In [ ]:
# Build the new datasets
train_lst = []
val_lst = []

for i in range(len(data["train"]["0"])):
  obs = data["train"]["0"][i]
  new_text = obs.split("Context:\n")[0]
  new_text += "Here are some examples:"
  new_text += select_examples(i, data["train"]["0"], "train")
  new_text += "\nContext\n"
  new_text += obs.split("Context:\n")[-1]
  train_lst.append(new_text)

for i in range(len(data["test"]["0"])):
  obs = data["test"]["0"][i]
  new_text = obs.split("Context:\n")[0]
  new_text += "Here are some examples:"
  new_text += select_examples(i, data["test"]["0"], "val")
  new_text += "\nContext\n"
  new_text += obs.split("Context:\n")[-1]
  val_lst.append(new_text)

In [ ]:
print(train_lst[0])

<s>[INST]Imagine being a Knowledge Graph constructor from unstructured text.
Following the schema provided, extract all the triples you can find in the text.

Schema:
(Legislator)-[REPRESENTS]->(StateCode)
(Legislator)-[IS_MEMBER_OF]->(Party)
(Legislator)-[ELECTED_TO]->(Body)
(BillId)-[SPONSORED_BY]->(Legislator)
(Legislator)-[VOTED_ON]->(BillId)
(BillId)-[REFERRED_TO]->(Committee)
(Legislator)-[SERVES_ON]->(Committee)
(BillId)-[DEALS_WITH]->(Subject)

Here are some examples:
Context:
"Coming up today, we have the hot-off-the-press update on bill hr2804-114, which deals with state and local government operations. This bill has a long list of sponsors, including Mike Quigley, Zoe Lofgren, Matt Cartwright, Don Beyer, Alan Lowenthal, Marc Veasey, Jared Huffman, and Raúl Grijalva. Stay tuned for more details as they emerge!"

Extracted Triples:
(s1400-114) - [DEALS_WITH] -> (State and local government operations)
(hr2804-114) - [SPONSORED_BY] -> (Mike Quigley )
(hr2804-114) - [SPONSORED_BY

In [ ]:
# The length of the new dataset corresponds to the length of the previous ones
print("length of training list: " + str(len(train_lst)))
print("length of val list: " + str(len(val_lst)))

length of training list: 2400
length of val list: 400


## Save to parquet
And later upload to HuggingFace

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd

In [ ]:
df_train = pd.DataFrame(train_lst)
df_val = pd.DataFrame(val_lst)
# Save train and validation into parquet format (and manually upload to huggingFace: Giardooo/KG_constructor_FS)
table = pa.Table.from_pandas(df_train)
pq.write_table(table, 'train-00000-of-00001.parquet')
table = pa.Table.from_pandas(df_val)
pq.write_table(table, 'test-00000-of-00001.parquet')